In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
from lxml import etree

In [3]:
main_url = 'https://kw-aruba.com/'
sub_links = ["listings/for-sale/residential", "listings/for-sale/condominium-townhouse",
             "listings/for-sale/land", "listings/for-sale/commercial", "listings/for-rent/residential",
             "listings/for-rent/condominium-townhouse", "listings/for-rent/land", "listings/for-sale/commercial"]

In [4]:
def join_links(main, sub):
    links  = []
    for sub in sub_links:
        url = main_url + sub
        links.append(url)
    return links
links = join_links(main_url, sub_links)

In [5]:
links

['https://kw-aruba.com/listings/for-sale/residential',
 'https://kw-aruba.com/listings/for-sale/condominium-townhouse',
 'https://kw-aruba.com/listings/for-sale/land',
 'https://kw-aruba.com/listings/for-sale/commercial',
 'https://kw-aruba.com/listings/for-rent/residential',
 'https://kw-aruba.com/listings/for-rent/condominium-townhouse',
 'https://kw-aruba.com/listings/for-rent/land',
 'https://kw-aruba.com/listings/for-sale/commercial']

In [6]:
def get_all_url(li):
    all_links = []
    for link in li:
        req = requests.get(link)
        soup = BeautifulSoup(req.content, "html.parser")
        anchors = soup.find('div', {'class': 'properties-grid'})
        for anchor in anchors:
            links  = anchor.find_all('a', href = True)
            for link in links:
                urls = main_url+ link.get('href')
                #print(urls)
                all_links.append(urls)
    return all_links
path = get_all_url(links)

In [9]:
def get_details(urls):
    property_list = []
    for link in urls:
        req = requests.get(link)
        soup = BeautifulSoup(req.content, "html.parser")
        dom = etree.HTML(str(soup))
        name = soup.find('span', class_ = "headline__inner").text
        location = soup.find('div', class_ = "page-property-details__location").text
        price = soup.find('div', class_ = "page-property-details__price").text.strip().split(" ")[1]
        rooms = soup.find("td", text="Bedrooms").find_next_sibling("td").text
        full_bathrooms = soup.find("td", text="Full bathrooms").find_next_sibling("td").text
        half_bathrooms = soup.find("td", text="Half bathrooms").find_next_sibling("td").text
        pool = soup.find("td", text="Pool").find_next_sibling("td").text
        link = link
        try:
            property_status = soup.find('div', class_ = "page-property-details__status").text
            property_type = soup.find('div', class_ = "page-property-details__type").text.split("-")[1]
        except AttributeError:
            property_status = None
            property_type = None
        lat = dom.xpath('//*[@id="map"]/script[1]')[0].text.strip().split('{')[2].split(',')[0].split(':')[1].strip()
        long = dom.xpath('//*[@id="map"]/script[1]')[0].text.strip().split('{')[2].split(',')[1].split(':')[1].strip().replace("}", "")

        prop = {
            'name': name,
            'location': location,
            'property_status': property_status,
            'property_type': property_type,
            'price': price,
            'bedrooms': rooms,
            'bathrooms' : full_bathrooms,
            'Pool':pool,
            'Latitude': lat,
            'Longitude': long,
            'link': link
        }
        property_list.append(prop)
        print('Saving:', prop['name'])
        
    df = pd.DataFrame(property_list)
    print(df.head(20))
    df.to_csv('kw_aruba.csv', header=True, index=False)

In [10]:
get_details(path)

Saving: L. G. SMITH BLVD - MALMOK
Saving: Rooi Santo 1-A
Saving: Opal 125
Saving: Oranjestad Luxury Villa
Saving: Opal 120
Saving: Westpunt 12-C
Saving: Seroe Janchi 31
Saving: Opal 138
Saving: Altovista 37-G
Saving: Opal 133
Saving: Caya di Oro 52 Tierra del Sol
Saving: Alto Vista 45-A
Saving: Villa Safir
Saving: Koyari Boutique Apartments
Saving: Villa Esmeralda
Saving: Safir 39
Saving: Monaco Vacation House
Saving: Villa Westpunt
Saving: Paradera 104-C
Saving: Caya Frere Johannes 4
Saving: Villa Casa Blanca
Saving: Tuscany Villas
Saving: Seroe Blanco 54-A
Saving: Palm Beach 33-A
Saving: Alto Vista Villa Park


ConnectionError: HTTPSConnectionPool(host='kw-aruba.com', port=443): Max retries exceeded with url: //listings/ponton-34g (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000022EAAE93580>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))